In [1]:
%%configure -f
{
    "numExecutors": 50, "executorCores": 3, "executorMemory": "8gb", "driverMemory": "2g"
}

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import healpy
from pyspark.sql import Row
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import LongType

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1637717202829_0002,pyspark,idle,Link,Link,arik,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
fire = spark.read.parquet('hdfs:///data/apogee_fire/m12f_lsr0/')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
def healpix(nside: pd.Series, ra: pd.Series, dec: pd.Series) -> pd.Series:
    theta = np.pi/2 - np.radians(dec.array)
    phi = np.radians(ra.array)
    nside = nside.iloc[0]
    return pd.Series(healpy.ang2pix(nside, theta, phi, nest=True))

healpix_udf = pandas_udf(healpix, returnType=LongType())
spark.udf.register('healpix', healpix_udf)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<function healpix at 0x7efcd1e53ca0>

In [5]:
fire.createOrReplaceTempView('fire')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
with_hp = spark.sql('''
SELECT *, healpix(8, ra, dec) as hp8, healpix(64, ra, dec) as hp64
FROM fire
''')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
with_hp.write.mode('overwrite').format('parquet') \
    .partitionBy('hp8') \
    .saveAsTable('apogee_fire_m12f_lsr0', path='hdfs:///user/arik/apogee_fire_m12f_lsr0_p-withhp')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
radecbin = spark.sql('''
SELECT ra, dec, count(*) FROM 
(SELECT CAST(ra AS INT) ra, CAST(dec AS INT) dec FROM fire)
GROUP BY ra, dec''').toPandas()

In [ ]:
radecbin

In [ ]:
plt.hexbin(radecbin['ra'], radecbin['dec'], C=radecbin['count(1)'], bins='log')

In [ ]:
%matplot plt

In [ ]:
with_hp.createOrReplaceTempView('with_hp8')

In [ ]:
hp8bin = spark.sql('''SELECT hp8, count(*) N from with_hp8 GROUP BY hp8 ORDER BY hp8 ASC''').toPandas()

In [9]:
fhp.createOrReplaceTempView('fhp')
hp8bin = spark.sql('''SELECT hp8, count(*) N from fhp GROUP BY hp8 ORDER BY hp8 ASC''').toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
import healpy

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
healpy.visufunc.mollview(hp8bin['N'], nest=True, norm='log')
healpy.graticule(coord='E')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
%matplot

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Interpreter died:

/tmp/nm-local-dir/usercache/arik/appcache/application_1637717202829_0001/container_1637717202829_0001_01_000001/pyspark.zip/pyspark/context.py:264: RuntimeWarning: Failed to add file [file:///opt/spark/python/lib/pyspark.zip] specified in 'spark.submit.pyFiles' to Python path:
  /tmp/nm-local-dir/usercache/arik/appcache/application_1637717202829_0001/container_1637717202829_0001_01_000001/tmp
  /tmp/nm-local-dir/usercache/arik/appcache/application_1637717202829_0001/spark-b99172fc-bb4d-4fb8-879f-d9cb7931d37f/userFiles-6538335a-b9a0-48c0-a27f-110f59d37be4
  /tmp/nm-local-dir/usercache/arik/appcache/application_1637717202829_0001/container_1637717202829_0001_01_000001/pyspark.zip
  /tmp/nm-local-dir/usercache/arik/appcache/application_1637717202829_0001/container_1637717202829_0001_01_000001/py4j-0.10.9-src.zip
  /usr/lib/python39.zip
  /usr/lib/python3.9
  /usr/lib/python3.9/lib-dynload
  /usr/local/lib/python3.9/dist-packages
  /usr/lib/python3/dist-packages
  warnin

In [ ]:
hp32bin = spark.sql('''SELECT hp64, count(*) N from (SELECT healpix(32, ra, dec) as hp64 FROM fire) GROUP BY hp64 ORDER BY hp64 ASC''').toPandas()